<a href="https://colab.research.google.com/github/erdalbucan/Colab/blob/main/Preprosesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [4]:
# makine ogrenmesi FORMUL  y = m1x1 + m2x2 + mnxn + b

In [ ]:
# https://www.kaggle.com/datasets/jacklacey/audi-a1-listings/data adresinden Audi_A1_listings.csv datadini indirdik
#pd.read_cvs ile dizindeki audi.csv dosyamizi okuyup df degiskenine taniimliyoruz
df = pd.read_csv('audi.csv')
df.head(3)

In [ ]:
# makine ogrenmesi icin ihtiyacimiz olmayan sutunlari temizleyelim
#bu datada fiyat tahmini icin araba linklerine ranklara ihtiyacim yok
df = df.drop(columns= ['index','href','MileageRank','PriceRank','PPY','PPYRank','Score'])
df.head(3)

In [ ]:
# Type , Transmission, Fuel gibi veriler KATEGORISEL rakamsal veriler ise NUMERIK verilerdir.

# y = mx + b  Y =tahmin edilecek veri x = etki katsayisi m = etki eden faktor (dizel, motor, yil vb. her biri ayri m1x1 m2x2 mnxn) b = sabit
# Data mizi Numerik veriye donusturmemiz gerekiyor.

# Datamizin neleri numerige cevirmemiz gerektigini ogrenmek icin pd.info()
df.info()
#Fload ve int harici butun verilerimiz Kategorisel veri imis

In [ ]:
# oncelikle sutun adlarini degistirmekle baslayalim
# oncelikle Engin sutununu NUmerik veriye donustirmekle baslayalim
df.columns=['yil','kasa','mil','motor','ps','vites','yakit','sahip','fiyat']
df.head(3)

In [ ]:
# motor sutundaki L ibarelerini str.replace komutuyla degistiriyorum
df['motor'] = df['motor'].str.replace('L','')
df.head(3)
#numerige benzedi fakat sutun henuz object olarak tanimli. int yada fload a cevirmek gerekiyor

In [ ]:
#numerige donuzmeye uygunlastirdigimiz motor sutunumuzu  to_numeric comutuyla numerige donusturuyoruz
df['motor'] = pd.to_numeric( df ['motor'] )
df['motor'].head(3)
# object olan motor sutunumuz artik fload

In [ ]:
# simdi direk kategorisel verileri numerige cevirecegiz get_dummies comutuyla
df= pd.get_dummies(df,columns=['kasa','vites','yakit'],drop_first=True) # drop_first=True iki veri yi teke cevir- yani oto vites ile man vites ayri olmaasin. oto vites 0 ise man vites demektir
df.head(5)
# Artik verimiz model olusturmak icin hazir. butun sutunlari NUMERIC

In [ ]:
# datamiz hazir. makine ogrenmesine gecebiliriz
# oncelikle x ve y mizi tanimliyoruz
# y = mx+b y= tahmin etmek istedigimiz veri = fiyat
y = df[['fiyat']]
# x diger butun etkenler x
x = df.drop('fiyat', axis = 1 ) # fiyat haricinde hepsi x olsun anlamina gelir yada x = df[['yil','motor',vs, vs]] yazabilir
df.head()

In [ ]:
# modelimizi olusturuyoruz
lm = LinearRegression()
model = lm.fit(x,y) #modeli fit ediyoruz ogretiyoruz

In [ ]:
# lm miz ogrendi, simdi bir tahmin yaptiralim predict comutuyla
model.predict([[2017,50000,1.6,103.550296,1,3800,0,1]])#yil	mil	motor	ps	sahip	FIYAT=X=YAZMA	ppy	vites_Manual	yakit_Petrol
# bu veri ye gore tahminimiz 14932 euro

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[14931.94147347]])

In [ ]:
# tahminler ne derece dogru
model.score(x,y)
# % 96 civarinda dogru tahmin ediyormus

0.9595295772839731

SyntaxError: invalid syntax (<ipython-input-105-7807c62fd889>, line 1)